In [95]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime
import time
# from statistics import multimode
import seaborn as sns
import plotly.express as px
from statsmodels.graphics.factorplots import interaction_plot
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats

### 1 Read in data

In [96]:
data_M_raw=pd.read_csv("race_Males.txt",sep="\t", encoding='ISO-8859-1')
data_F_raw=pd.read_csv("race_Females.txt",sep="\t")


In [97]:
print(data_F_raw.shape,data_F_raw.dtypes)
print('########')
print(data_M_raw.shape,data_M_raw.dtypes)

(1105, 9) Place         int64
Div/Tot      object
Num           int64
Name         object
Ag          float64
Hometown     object
Gun Tim      object
Net Tim      object
Pace         object
dtype: object
########
(1265, 9) Place         int64
Div/Tot      object
Num           int64
Name         object
Ag          float64
Hometown     object
Gun Tim      object
Net Tim      object
Pace         object
dtype: object


### 2 Data cleaning
#### Missing data

In [98]:
# check missing data
def missing_data(data):
    '''check missing data'''
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

print(missing_data(data_F_raw))
print('########')
print(missing_data(data_M_raw))

          Total   Percent
Div/Tot       2  0.180995
Ag            1  0.090498
Pace          0  0.000000
Net Tim       0  0.000000
Gun Tim       0  0.000000
Hometown      0  0.000000
Name          0  0.000000
Num           0  0.000000
Place         0  0.000000
########
          Total   Percent
Div/Tot       2  0.158103
Ag            1  0.079051
Pace          0  0.000000
Net Tim       0  0.000000
Gun Tim       0  0.000000
Hometown      0  0.000000
Name          0  0.000000
Num           0  0.000000
Place         0  0.000000


Div/Tot and Ag have a small amount of missing data in both files

In [99]:
# print na entries
print(data_M_raw.loc[data_M_raw['Div/Tot'].isna(),:])
print('################################################')
print(data_F_raw.loc[data_F_raw['Div/Tot'].isna(),:])

     Place Div/Tot   Num            Name   Ag        Hometown Gun Tim Net Tim  \
525    526     NaN  2003      Chris Nash  0.0  Brookeville MD   51:04   49:35   
724    725     NaN  1999  Alan Nadelbach  NaN     Columbia MD   54:16   52:56   

     Pace  
525  7:59  
724  8:32  
################################################
     Place Div/Tot   Num             Name   Ag         Hometown  Gun Tim  \
964    965     NaN   201    Carla Sabloff  0.0        Mclean VA  1:13:38   
979    980     NaN  2843  Susan Rasmussen  NaN  Gaithersburg MD  1:14:51   

     Net Tim   Pace  
964  1:08:29  11:02  
979  1:09:34  11:12  


from above we can see that the 4 racers have both missing/wrong age and Div/Tot, so we should remove them from the dataset

In [100]:
data_F_raw = data_F_raw.loc[data_F_raw['Div/Tot'].notna(),:].copy().reset_index()
data_M_raw = data_M_raw.loc[data_M_raw['Div/Tot'].notna(),:].copy().reset_index()

#### Clean guntime and Hometown

In [101]:
# find out rows with messed up guntime in MA_Exer_PikesPeak_Females
print(len(data_F_raw.loc[data_F_raw['Gun Tim'].str.match(r'[A-Z]'),'Gun Tim']))
mess = data_F_raw.loc[data_F_raw['Gun Tim'].str.match(r'[A-Z]'),:]

print(mess.head())


137
    index  Place Div/Tot   Num                  Name    Ag         Hometown  \
30     30     31   5/173  2153          Emily Pierce  30.0  Silver Spring M   
33     33     34    7/86  1852  Marycolle Mcallister  22.0  Silver Spring M   
37     37     38   7/247  1561     Christina Kichula  35.0  Silver Spring M   
45     45     46  10/247  2727        Paige Waterman  38.0  Silver Spring M   
58     58     59    7/26  1655      Krisztina Larson  19.0  Silver Spring M   

      Gun Tim Net Tim  Pace  
30  D   42:03   41:56  6:45  
33  D   43:24   42:19  6:49  
37  D   44:00   42:58  6:55  
45  D   45:01   43:51  7:04  
58  D   46:03   44:59  7:15  


In [102]:
# revise hometown
data_F_raw.loc[:,'Hometown'] = np.where(data_F_raw['Gun Tim'].str.match(r'^[A-Z]'),\
            data_F_raw['Hometown']+data_F_raw['Gun Tim'].apply(lambda x:x[0]),data_F_raw['Hometown'])

In [103]:
# revise gun time
data_F_raw.loc[:,'Gun Tim'] = data_F_raw.loc[:,'Gun Tim'].apply(lambda x:x[-8:])

#### Clean age error
There are a few racers have age of -1 and 1, which maybe mistakes, we can find their devisions and replace them with the median age of their division

In [104]:
data_F_raw.loc[data_F_raw.Ag ==-1,: ]

,index,Place,Div/Tot,Num,Name,Ag,Hometown,Gun Tim,Net Tim,Pace
382,382,383,4/15,1917,Stefanie Merritt,-1.0,Alexandria VA,55:33,54:22*,8:45
1006,1008,1009,15/15,1778,Susan Mackey,-1.0,North Potomac MD,1:16:36,1:11:18*,11:29


In [105]:
data_F_raw.loc[data_F_raw.Ag ==1,: ]


,index,Place,Div/Tot,Num,Name,Ag,Hometown,Gun Tim,Net Tim,Pace
704,704,705,10/15,1726,Arminda Lima,1.0,Silver Spring MD,1:03:47,1:00:07,9:41


In [106]:
data_M_raw.loc[data_M_raw.Ag ==-1,: ]

,index,Place,Div/Tot,Num,Name,Ag,Hometown,Gun Tim,Net Tim,Pace
789,791,792,15/28,1555,Jon Kesler,-1.0,Silver Spring MD,59:20,54:06,8:43
1207,1209,1210,26/28,556,Chris Barr,-1.0,Germantown MD,1:15:39,1:10:30,11:21


In [107]:
data_M_raw.loc[(data_M_raw['Div/Tot'].str.match('.*/28'))&(data_M_raw["Ag"]<0),"Ag"] = \
            np.median(data_M_raw.loc[data_M_raw['Div/Tot'].str.match('.*/28')&(data_M_raw["Ag"]>0),"Ag"])
data_F_raw.loc[(data_F_raw['Div/Tot'].str.match('.*/15'))&(data_F_raw["Ag"]<2),"Ag"] = \
            np.median(data_F_raw.loc[data_F_raw['Div/Tot'].str.match('.*/28')&(data_F_raw["Ag"]>2),"Ag"])


#### Combining Female and Male Data

In [108]:
data_F_raw['gender'] = 'F'
data_M_raw['gender'] = 'M'
df = data_M_raw.append(data_F_raw,ignore_index=True)
df.shape

(2366, 11)

#### Add a new column as division based on the cleaned age

In [109]:
df["div"] = np.where(df["Ag"]<15,"0-14",np.where(df["Ag"]<20,"15-19",df["Ag"].apply(lambda x:f"{int(x)//10}0-{int(x)//10}9")))

In [110]:
df[["div","Place"]].groupby(['div'],as_index=False).count().rename(columns={"Place":"Count"})

,div,Count
0,0-14,40
1,15-19,71
2,20-29,372
3,30-39,766
4,40-49,698
5,50-59,312
6,60-69,93
7,70-79,12
8,80-89,2


#### Clean up the time columns
Change them as minutes

In [111]:
 # Remove * and # symbols from net time column
df.loc[:,'Net Tim'] = df.loc[:,'Net Tim'].apply(lambda x: x.strip('#*'))
# drop index column and the Div/Tot since we create the new div, also drop ori gun time and net time
df = df.drop(['index'], axis=1)
df = df.drop(['Div/Tot'], axis=1)
df.head()

,Place,Num,Name,Ag,Hometown,Gun Tim,Net Tim,Pace,gender,div
0,1,9,Joshua Kemei,22.0,Kenya .,28:48,28:47,4:38,M,20-29
1,2,35,George Kirwa Misoi,22.0,Kenya .,29:11,29:10,4:42,M,20-29
2,3,3,Jacob Frey,24.0,Oakton VA,29:38,29:37,4:46,M,20-29
3,4,49,Gurmessa Kumsa,27.0,Oxon Hill MD,29:46,29:46,4:48,M,20-29
4,5,31,Mike Baird,24.0,Ardmore PA,30:31,30:30,4:55,M,20-29


In [112]:
# add new columns as minutes from time columns
df['gun_s'] = df['Gun Tim'].apply(lambda x: np.sum([int(i)*60**(x.count(":")-j-1) for j,i in enumerate(x.split(":"))]))
df['net_s'] = df['Net Tim'].apply(lambda x: np.sum([int(i)*60**(x.count(":")-j-1) for j,i in enumerate(x.split(":"))]))
df['pace_s'] = df['Pace'].apply(lambda x: np.sum([int(i)*60**(x.count(":")-j-1) for j,i in enumerate(x.split(":"))]))

In [113]:
df.head()

,Place,Num,Name,Ag,Hometown,Gun Tim,Net Tim,Pace,gender,div,gun_s,net_s,pace_s
0,1,9,Joshua Kemei,22.0,Kenya .,28:48,28:47,4:38,M,20-29,28.800000,28.783333,4.633333
1,2,35,George Kirwa Misoi,22.0,Kenya .,29:11,29:10,4:42,M,20-29,29.183333,29.166667,4.700000
2,3,3,Jacob Frey,24.0,Oakton VA,29:38,29:37,4:46,M,20-29,29.633333,29.616667,4.766667
3,4,49,Gurmessa Kumsa,27.0,Oxon Hill MD,29:46,29:46,4:48,M,20-29,29.766667,29.766667,4.800000
4,5,31,Mike Baird,24.0,Ardmore PA,30:31,30:30,4:55,M,20-29,30.516667,30.500000,4.916667


In [114]:
# drop ori gun time and net time, pace
df = df.drop(['Gun Tim'], axis=1)
df = df.drop(['Net Tim'], axis=1)
df = df.drop(['Pace'], axis=1)
df.head()

,Place,Num,Name,Ag,Hometown,gender,div,gun_s,net_s,pace_s
0,1,9,Joshua Kemei,22.0,Kenya .,M,20-29,28.800000,28.783333,4.633333
1,2,35,George Kirwa Misoi,22.0,Kenya .,M,20-29,29.183333,29.166667,4.700000
2,3,3,Jacob Frey,24.0,Oakton VA,M,20-29,29.633333,29.616667,4.766667
3,4,49,Gurmessa Kumsa,27.0,Oxon Hill MD,M,20-29,29.766667,29.766667,4.800000
4,5,31,Mike Baird,24.0,Ardmore PA,M,20-29,30.516667,30.500000,4.916667


In [115]:
# get cleaned data by gender
df_f = df.loc[df.gender=='F',:].copy()
df_m = df.loc[df.gender=='M',:].copy()

In [116]:
print(df.shape[0])
print(df_f.shape[0])
print(df_m.shape[0])
print(min(df.Ag))
print(max(df.Ag))

2366
1103
1263
9.0
84.0


In [117]:
# save cleaned data
df.to_csv('cleaned_df.csv', index=False)